# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 311560 # по умолчанию вписан мой номер

2022-12-12 08:52:10.518170


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Ремонт', 'Предсказатели будущего', 'Ювелирные изделия']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Одежда (также нужно искать "одежды")', 'Магазины косметики и Салоны красоты', 'Прокат автомобилей', 'Видеоигры (искать надо по "видеоигр")', 'Программирование', 'Предсказатели будущего']
В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 12 признаков, то есть шесть категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
from torch.autograd import Variable
from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
transactions = pd.read_csv("/content/sample_data/transactions.csv", nrows=1000000, sep=",")
gender_train=pd.read_csv("/content/sample_data/gender_train.csv", sep=",")
tr_mcc_codes=pd.read_csv("/content/sample_data/tr_mcc_codes.csv", sep=";")

In [ ]:
descriptions = [['одежд'], ['косметик', 'красот'], 
                ['прокат авто', 'автомобил'], ['видеоигр'], ['програм'],
                ['будущ']]


def str_contains_word_in_list(string):
    counter = 0
    for category in descriptions:
        for w in category:
            if w in string:
                return True, counter
        counter += 1
    return False, None


checks, category = zip(*tr_mcc_codes.mcc_description.apply(str_contains_word_in_list))
tr_mcc_codes['category'] = category
tr_category_codes = tr_mcc_codes.loc[list(checks)]
tr_category_codes['category'] = tr_category_codes['category'].apply(int)
tr_category_codes.category.value_counts()

<ipython-input-56-e067bd4401ba>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_category_codes['category'] = tr_category_codes['category'].apply(int)


0    8
2    3
4    2
1    2
3    2
5    1
Name: category, dtype: int64

In [ ]:
df = transactions.merge(tr_category_codes, on='mcc_code')
df.category.value_counts()

0    11005
1     6126
3     1141
4      415
2       65
5       46
Name: category, dtype: int64

## Вопросы к заданию 1 (6 баллов)

1. Для каждой категории выведите соответствующие ей mcc-коды с описаниями.
2. Существуют ли пользователи, у которых значения всех признаков не определены?
3. Выведите id пользователя, у которого самое большое количество непустых признаков.

## Ответы:

### Создание feature-DF

In [ ]:
multyindex = df.groupby(['customer_id', 'category'])['amount'].agg(['count']).unstack().reset_index()

new_df = pd.DataFrame({f'count_category_{c_name}' 
                       : multyindex['count'][c_name].tolist()
                       for c_name in multyindex['count'].columns.tolist()})
new_df['customer_id'] = multyindex['customer_id'].tolist()

in_come = df[df['amount'] > 0].groupby(['customer_id', 'category'])['amount'].agg(['sum']).unstack().reset_index()

new_df = new_df.merge(in_come, on='customer_id', how='left')

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


### Подготовка DF

In [ ]:
new_df = new_df.merge(gender_train, on='customer_id')
new_df

,count_category_0,count_category_1,count_category_2,count_category_3,count_category_4,count_category_5,customer_id,"(sum, 0)","(sum, 1)","(sum, 2)","(sum, 4)",gender_x,gender_y
0,8.0,5.0,NaN,NaN,NaN,NaN,115617,NaN,NaN,NaN,NaN,1,1
1,8.0,2.0,NaN,NaN,NaN,NaN,189257,NaN,NaN,NaN,NaN,1,1
2,10.0,6.0,NaN,NaN,NaN,NaN,293048,NaN,NaN,NaN,NaN,0,0
3,2.0,NaN,NaN,NaN,1.0,NaN,361458,NaN,NaN,NaN,NaN,1,1
4,4.0,NaN,NaN,NaN,NaN,NaN,496551,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2.0,NaN,NaN,NaN,2.0,NaN,99302313,NaN,NaN,NaN,NaN,1,1
740,10.0,3.0,NaN,NaN,NaN,NaN,99412328,NaN,NaN,NaN,NaN,0,0
741,NaN,2.0,NaN,NaN,NaN,NaN,99683064,NaN,NaN,NaN,NaN,0,0
742,1.0,NaN,NaN,NaN,NaN,NaN,99717689,NaN,NaN,NaN,NaN,1,1


### Вопрос 1

In [ ]:
tr_category_codes.sort_values(by='category')

,mcc_code,mcc_description,category
78,5697,"Услуги по переделке, починке и пошиву одежды",0
39,5137,"Мужская, женская и детская спец-одежда",0
69,5611,"Мужская одежда и аксессуары, включая одежду дл...",0
70,5621,Готовая женская одежда,0
72,5641,"Детская одежда, включая одежду для самых мален...",0
74,5655,"Спортивная одежда, одежда для верховой езды и ...",0
77,5691,Магазины мужской и женской одежды,0
79,5699,Различные магазины одежды и аксессуаров,0
135,7230,Салоты красоты и парикмахерские,1
116,5977,Магазины косметики,1


In [ ]:
new_df[new_df['customer_id'] == 293048].dropna(axis=1)

,count_category_0,count_category_1,customer_id,gender
2,10.0,6.0,293048,0


### Вопрос 2

In [ ]:
def all_none_check(c_id):
  return new_df[new_df['customer_id'] == c_id].dropna(axis=1).columns.__len__() == 1


new_df.loc[new_df.customer_id.apply(all_none_check)]

,count_category_0,count_category_1,count_category_2,count_category_3,count_category_4,count_category_5,customer_id,"(sum, 0)","(sum, 1)","(sum, 2)","(sum, 4)",gender


### Вопрос 3

In [ ]:
def max_count(c_id):
  return new_df[new_df['customer_id'] == 115617].dropna(axis=1).columns.__len__() - 1

In [ ]:
new_df['count'] = new_df.customer_id.apply(max_count)
new_df[['customer_id', 'count']]

AttributeError: ignored

не успел дописать

# Задание 2 (3 балла)

Модель - многослойная нейронная сеть минимум с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Настроить параметры модели.

## Вопросы к заданию 2

1. Перечислите все гиперпараметры и их значения вот в таком формате (значения приведены примерные, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.0001
* функция активация на скрытых слоях: ReLU
* и т.д.

2. Сколько у вашей модели параметров?

## Ответы:

# Задание 3 (2 балла)


Проверить качество модели не менее, чем на трёх разных метриках. 

## Вопросы к заданию 3

1. Выведите значения метрик.
2. Как улучшить модель?
3. Для чего может понадобиться предсказание пола клиента по его транзакциям?

## Ответы:

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2022-12-12 08:22:52.089295
